In [1]:
import h5py as h5
#from h5glance import H5Glance
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import gvar as gv
import pandas as pd
import os 
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib notebook
#import h5ls
import lsqfit
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import yaml
from pathlib import Path
import importlib

In [2]:
#import local files
#sys.path.append("../")
import gmo_fitter as gmo 
import i_o 
import gmo_fit_analysis as fa
import load_data_priors as ld
import mass_relations as ma
matplotlib.rcParams['figure.figsize'] = [10, 8]

In [3]:
from input_files import a12m180L


p_dict = a12m180L.p_dict
abbr = p_dict['abbr']

In [4]:
import platform
if platform.system() == 'Darwin':
    file = '/Users/grantdb/lqcd/data/c51_2pt_octet_decuplet.h5'
else:
    file = '/home/gmoney/lqcd/data/c51_2pt_octet_decuplet.h5'
with h5.File(file,"r") as f:
    ensembles = {}
    ensembles = list(f.keys())
    # print(ensembles)

In [5]:
# hacky way to initialize all raw corr data 
# importlib.reload(ld)
corr = {}
for key in p_dict['gmo_states']:
    corr[key] = ld.get_raw_corr(file,p_dict['abbr'],particle=key)
# print(corr)
nucleon_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='proton')
lam_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='lambda_z')
xi_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='xi_z')
sigma_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='sigma_p')
piplus_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='piplus')
kplus_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='kplus')
delta_corr = ld.get_raw_corr(file,p_dict['abbr'],particle='delta_pp')
gmo_ratio_raw = ld.G_gmo(file,p_dict['abbr'],log=False)
print(gmo_ratio_raw)

# nucleon_bs = fa.bs_to_gvar(nucleon_corr,bs_N=100)
# lam_bs = fa.bs_to_gvar(lam_corr,bs_N=100)
# xi_bs = fa.bs_to_gvar(xi_corr,bs_N=100)
# sigma_bs = fa.bs_to_gvar(sigma_corr,bs_N=100)
# gmo_bs = fa.bs_to_gvar(gmo_ratio_raw,bs_N=100)


# eff_mass = ma.gmo_eff_mass(gmo_out=gmo_ratio_raw,dt=1)
# gmo_ratio_raw
# nucleon_corr
# import tqdm
# bs_M = nucleon_corr['PS'].shape[0] # bs_M would be the same if you used 'ps' instead
# bs_N = 100 # In real-world cases, this would probably be much larger
# bs_list = np.random.randint(low=0, high=bs_M, size=(bs_N, bs_M))
# for j in tqdm.tqdm(range(bs_N), desc='bootstrap'):
#     for key in p_dict['gmo_states']:

#         gv.switch_gvar() 
#         bs_nucleon = fa.resample_corr(raw_corr=corr[key], bs_list=bs_list,n=j )

#         print(bs_nucleon)


{'SS': array([0.6226(19), 0.605353(88), 0.60475(16), 0.60406(21), 0.60323(25),
       0.60219(29), 0.60111(36), 0.60007(47), 0.59937(70), 0.5983(10),
       0.5963(14), 0.5938(20), 0.5904(27), 0.5857(39), 0.5814(58),
       0.5783(86), 0.568(13), 0.553(17), 0.545(24), 0.561(37), 0.585(61),
       0.62(10), 0.75(24), 1.6(2.7), nan +- nan, nan +- nan, nan +- nan,
       0.8(1.1), 0.43(25), 0.39(25), 0.6(1.1), nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, 1.5(4.3),
       nan +- nan, nan +- nan, nan +- nan, 0.05(31), -0.12(30), -0.54(85),
       -1.0(1.9), -0.8(1.9), -0.29(89), 0.28(44), 0.54(34), 0.49(25),
       0.50(23), 0.70(52), nan +- nan, 98(109369), 1.06(73), 0.80(20),
       0.743(79), 0.715(31), 0.683(15), 0.6600(81), 0.6379(46),
       0.6203(24), 0.6095(11), 0.60597(26)], dtype=object), 'PS': array([0.6428(48), 0.60554(14), 0.60492(30), 0.60422(39), 0.60342(39),
       0.60224(39), 0.60103(45), 0.59997(58), 0.59980(94), 0.5989(14),
       0.596

/home/gmoney/lqcd/gmo_relation/load_data_priors.py:55: RuntimeWarning: invalid value encountered in power
  * np.power(temp[('proton', smr)], -2/3)


In [6]:
prior_nucl = {}
prior = {}
states=p_dict['gmo_states_all']
newlist = [x for x in states]
for x in newlist:
    path = os.path.normpath("./priors/{0}/{1}/prior_nucl.csv".format(p_dict['abbr'],x))
    df = pd.read_csv(path, index_col=0).to_dict()
    for key in list(df.keys()):
        length = int(np.sqrt(len(list(df[key].values()))))
        prior_nucl[key] = list(df[key].values())[:length]
        # prior_nucl['gmo_E'] = list([np.repeat(gv.gvar('0.0030(27)'),8)])
    prior = gv.gvar(prior_nucl)
# prior_gmo = {}
# prior['gmo_E'] = gv.gvar('0.0030(27)') #this shouldnt be priored like this? #

In [10]:
importlib.reload(gmo)
importlib.reload(fa)
importlib.reload(a12m180L)
importlib.reload(ma)
model_type = 'simult_baryons'
# simult_gmo_bs = fa.fit_ensemble(t_range=p_dict
# ['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior,
# nucleon_corr_data=nucleon_bs,lam_corr_data=lam_bs, xi_corr_data=xi_bs,sigma_corr_data=sigma_bs,delta_corr_data=None,gmo_corr_data=None,
# piplus_corr_data=None,kplus_corr_data=None,model_type=model_type)
# # print(gmo_.get_fit().format(maxline=True))
# print(simult_gmo_bs)

simult_gmo = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64,states=p_dict['gmo_states'], n_states=p_dict['n_states'],prior=prior,
nucleon_corr_data=nucleon_corr,lam_corr_data=lam_corr, xi_corr_data=xi_corr,sigma_corr_data=sigma_corr,
delta_corr_data=None,gmo_corr_data=None,
piplus_corr_data=None,kplus_corr_data=None,model_type=model_type)

# print(gmo_.get_m_4(t=None))
fit_out = simult_gmo.get_fit()
print(fit_out)
simult_gmo.plot_effective_mass( t_plot_min=0,t_plot_max=15, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)
out_path = 'fit_results/{0}/'.format(p_dict['abbr'],model_type)

if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
ld.pickle_out(fit_out=fit_out, out_path=out_path,species='all')
corr_fits = {} # resulting fit params, need E0 for gmo study 
mass = {}
gmo_states = ['sigma','xi','proton','lam']
abbr =p_dict['abbr']

# for ens in ensembles:
mass[p_dict['abbr']] = {}
out_path = 'fit_results/{0}/'.format(p_dict['abbr'],model_type)
for corr in gmo_states:
    corr_fits = gv.load(out_path+'fit_params_all')
    mass[p_dict['abbr']][corr] = corr_fits['p'][corr+'_E0']

Least Square Fit:
  chi2/dof [dof] = 1.1 [54]    Q = 0.37    logGBF = 1286.3

Parameters:
        gmo_E0 0           0.3 (2.5)        [    0.3 (2.5) ]  
          z_PS 0   -4e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
               1    1e-21 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
          z_SS 0        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
               1        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
    sigma_z_PS 0       3.105(64)e-06        [ 0.0(3.3)e-06 ]  
               1        4.86(48)e-06        [ 0.0(3.3)e-06 ]  *
    sigma_z_SS 0        8.27(16)e-07        [ 4.4(4.4)e-07 ]  
               1        6.69(57)e-07        [ 4.4(4.4)e-07 ]  
      lam_z_PS 0       1.487(37)e-06        [ 0.0(3.3)e-06 ]  
               1        2.25(30)e-06        [ 0.0(3.3)e-06 ]  
      lam_z_SS 0       3.924(89)e-07        [ 4.4(4.4)e-06 ]  
               1        3.50(39)e-07        [ 4.4(4.4)e-06 ]  
   proton_z_PS 0        1.80(11)e-06        [ 0.0(3.3)e-05 ]  
               1        2.6

/home/gmoney/lqcd/gmo_relation/gmo_fit_analysis.py:468: RuntimeWarning: invalid value encountered in log
  return {key : 1/dt * np.log(nucleon_corr_gv[key] / np.roll(nucleon_corr_gv[key], -1))


<IPython.core.display.Javascript object>

FileNotFoundError: [Errno 2] No such file or directory: 'fit_results/a12m180L/fit_params_all'

In [ ]:
importlib.reload(gmo)
importlib.reload(fa)
importlib.reload(a12m180L)
importlib.reload(ma)
model_type = 'simult_baryons_gmo'
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior,
nucleon_corr_data=nucleon_corr,lam_corr_data=lam_corr, xi_corr_data=xi_corr,sigma_corr_data=sigma_corr,delta_corr_data=None,gmo_corr_data=gmo_ratio_raw,
piplus_corr_data=None,kplus_corr_data=None,model_type=model_type)
print(gmo_)
gmo_eff_mass = gmo_.get_gmo_effective(gmo_ratio=gmo_ratio_raw)
print(gmo_eff_mass)
gmo_.plot_gmo_effective_mass(effective_mass=gmo_eff_mass,t_plot_min=0,t_plot_max=20, model_type=model_type,fig_name='plots/{0}_{1}_eff'.format(abbr,model_type),show_fit=True)
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])

if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
ld.pickle_out(fit_out=fit_out, out_path=out_path,species='all')
corr_fits = {} # resulting fit params, need E0 for gmo study 
mass = {}
gmo_states = ['sigma','xi','proton','lam']
abbr =p_dict['abbr']

# for ens in ensembles:
mass[p_dict['abbr']] = {}
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
for corr in gmo_states:
    corr_fits = gv.load(out_path+'fit_params_all')
    mass[p_dict['abbr']][corr] = corr_fits['p'][corr+'_E0']
        
# print(mass)
gmo_raw =  ma.GMO(mass_dict=mass,file=file,abbr=abbr,baryons=True)
gmo_rln = gmo_raw.gmo_rln()
m_4 = gmo_raw.m_4()
centroid = gmo_raw.centroid()
delta_gmo = gmo_raw.gmo_violation()

Model Type:simult_baryons_gmo
	 N_{corr} = 2	

Fit results: 
	 GMO rln = 0.0049(15)
	 M_4 posterior =    -0.1616(69)
	 Centroid posterior =    0.7029(13)
	 lam_E0=  0.6739(16)
	 1st e.s:  0.399(28)
	 proton_E0=  0.5894(34)
	 1st e.s:  0.339(40)
	 xi_E0=  0.78188(93)
	 1st e.s:   0.449(18)
	 sigma_E0=  0.7356(19)
	 1st e.s:  0.447(27)
Least Square Fit:
  chi2/dof [dof] = 0.98 [90]    Q = 0.52    logGBF = 1854.5

Parameters:
        gmo_E0 0           0.3 (2.5)        [    0.3 (2.5) ]  
          z_PS 0   -8e-22 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
               1   -1e-21 +- 3.3e-06        [ 0.0(3.3)e-06 ]  
          z_SS 0        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
               1        4.4(4.4)e-07        [ 4.4(4.4)e-07 ]  
    sigma_z_PS 0       3.102(63)e-06        [ 0.0(3.3)e-06 ]  
               1        4.86(48)e-06        [ 0.0(3.3)e-06 ]  *
    sigma_z_SS 0        8.26(15)e-07        [ 4.4(4.4)e-07 ]  
               1        6.70(56)e-07        [ 4.4(4.4)e-07 ]  
     

/home/gmoney/lqcd/gmo_relation/gmo_fit_analysis.py:270: RuntimeWarning: invalid value encountered in log
  return {key : 1/dt * np.log(gmo_ratio[key] / np.roll(gmo_ratio[key], -1))


<IPython.core.display.Javascript object>

FileNotFoundError: [Errno 2] No such file or directory: 'fit_results/a12m180L/fit_params_all'

In [ ]:
importlib.reload(fa)
model_type='gmo_ratio'
# print(gmo_ratio_raw)
gmo_.plot_delta_gmo(correlators_gv = gmo_ratio_raw, t_plot_min=0,t_plot_max=16, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model_type = 'gmo_ratio'


In [ ]:
model_type='proton'
gmo_.plot_effective_mass( t_plot_min=5,t_plot_max=15, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

UnboundLocalError: local variable 'nucleon_corr_gv' referenced before assignment

In [ ]:
model_type = 'gmo'
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior, 
nucleon_corr_data=nucleon_corr,lam_corr_data=lam_corr, xi_corr_data=xi_corr,sigma_corr_data=sigma_corr,delta_corr_data=delta_corr, gmo_corr_data=
piplus_corr_data=None,kplus_corr_data=None,model_type=model_type)
print(gmo_.get_fit().format(maxline=True))
fit_out = gmo_.get_fit()
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
ld.pickle_out(fit_out=fit_out, out_path=out_path,species='all')



SyntaxError: invalid syntax (860948316.py, line 5)

In [ ]:
model_type = 'meson'
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior, 
nucleon_corr_data=None,lam_corr_data=None, xi_corr_data=None,sigma_corr_data=None,delta_corr_data=None,
piplus_corr_data=piplus_corr,kplus_corr_data=kplus_corr,model_type=model_type)
print(gmo_.get_fit().format(maxline=True))
fit_out = gmo_.get_fit()
# out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
# ld.pickle_out(fit_out=fit_out, out_path=out_path,species='all')

NameError: name 'fa' is not defined

In [ ]:
model_type='delta'
gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=16, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

In [ ]:
model_type='pi'
gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=25, model_type='pi', fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

<IPython.core.display.Javascript object>

UnboundLocalError: local variable 'gmo_ratio_corr' referenced before assignment

In [ ]:
model_type='kplus'
gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=25, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model_type='proton'
gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=15, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

In [ ]:
model_type='xi'
gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=15, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

In [ ]:
model_type='sigma'
gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=15, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

In [ ]:
model_type='lam'
gmo_.plot_effective_mass(t_plot_min=5,t_plot_max=15, model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True)

In [ ]:
# meson fits #
model_type = 'pi'
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior, 
nucleon_corr_data=None,lam_corr_data=None, xi_corr_data=None,sigma_corr_data=None,
piplus_corr_data=piplus_corr,kplus_corr_data=None,model_type=model_type)
print(gmo_.get_fit().format(maxline=True))
fit_out = gmo_.get_fit()
t = np.arange(6,20)
gmo_fit_data = gmo_._generate_data_from_fit(t=t,model_type=model_type)
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)

gmo_.plot_effective_mass(nucleon_corr_gv=gv.dataset.avg_data(piplus_corr) ,model_type=model_type, fig_name='plots/{0}_{1}'.format(abbr,model_type),show_fit=True,)


Least Square Fit:
  chi2/dof [dof] = 1.6e+03 [50]    Q = 0    logGBF = -39106

Parameters:
         gmo_E 0     0.003 (25)      [   0.003 (25) ]  
    delta_z_PS 0   0.0(1.5)e-05      [ 0.0(1.5)e-05 ]  
               1   0.0(1.5)e-05      [ 0.0(1.5)e-05 ]  
    delta_z_SS 0   2.0(2.0)e-06      [ 2.0(2.0)e-06 ]  
               1   2.0(2.0)e-06      [ 2.0(2.0)e-06 ]  
    sigma_z_PS 0   0.0(3.3)e-06      [ 0.0(3.3)e-06 ]  
               1   0.0(3.3)e-06      [ 0.0(3.3)e-06 ]  
    sigma_z_SS 0   4.4(4.4)e-07      [ 4.4(4.4)e-07 ]  
               1   4.4(4.4)e-07      [ 4.4(4.4)e-07 ]  
      lam_z_PS 0   0.0(1.5)e-05      [ 0.0(1.5)e-05 ]  
               1   0.0(1.5)e-05      [ 0.0(1.5)e-05 ]  
      lam_z_SS 0   2.0(2.0)e-06      [ 2.0(2.0)e-06 ]  
               1   2.0(2.0)e-06      [ 2.0(2.0)e-06 ]  
   proton_z_PS 0   0.0(3.3)e-06      [ 0.0(3.3)e-06 ]  
               1   0.0(3.3)e-06      [ 0.0(3.3)e-06 ]  
   proton_z_SS 0   9.4(9.4)e-07      [ 9.4(9.4)e-07 ]  
             

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#TODO this should run if provided a "fit is final, ready to write out results" #
gv.dump(gmo_fit_data,out_path+'meson_fit_data')
ld.pickle_out(fit_out=fit_out, out_path=out_path,species='meson')

'fit_results/a15m310/meson_fit_params'

In [ ]:
fit_state = p_dict['fit_state']              # this should not require manually changing corr data!!!
gmo_ = fa.fit_ensemble(t_range=p_dict
['t_range'],t_period=64, n_states=p_dict['n_states'],prior=prior, 
nucleon_corr_data=nucleon_corr,lam_corr_data=lam_corr, xi_corr_data=xi_corr,sigma_corr_data=sigma_corr,
piplus_corr_data=None,kplus_corr_data=None)
print(gmo_.get_fit().format(maxline=True))
fit_out = gmo_.get_fit()
t = np.arange(8-2, 15)
gmo_fit_data = gmo_._generate_data_from_fit(t=t,model_type='gmo')
out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
if os.path.exists(out_path):
    pass
else:
    os.mkdir(out_path)
gv.dump(gmo_fit_data,out_path+'fit_data')
fit_dump = {}
fit_dump['prior'] = fit_out.prior
fit_dump['p'] = fit_out.p
fit_dump['logGBF'] = fit_out.logGBF
fit_dump['Q'] = fit_out.Q
gv.dump(fit_dump,out_path+'fit_params')

KeyError: 'fit_state'

In [ ]:
model_type = 'sigma'
gmo_.plot_effective_mass(nucleon_corr_gv=sigma_corr_gv ,model_type=model_type, fig_name='a09m135_{0}'.format(model_type),show_fit=True,)


NameError: name 'sigma_corr_gv' is not defined

In [ ]:
gmo_.plot_stability(model_type='sigma',fig_name='a12m180L_{0}_stab'.format(p_dict['fit_state']))


<IPython.core.display.Javascript object>

((6, 15, 2), (6, 15, 2), (6, 15, 2), (3, 15, 2), (6, 15, 2))
Least Square Fit:
  chi2/dof [dof] = 17 [78]    Q = 4.2e-232    logGBF = 1269

Parameters:
    sigma_z_PS 0    6.061(32)e-07       [ 0.0(3.3)e-06 ]  
               1    4.139(18)e-06       [ 0.0(3.3)e-06 ]  *
    sigma_z_SS 0   1.7119(82)e-07       [ 4.4(4.4)e-07 ]  
               1    5.690(21)e-07       [ 4.4(4.4)e-07 ]  
      lam_z_PS 0    2.501(39)e-07       [ 0.0(3.3)e-06 ]  
               1     9.87(52)e-07       [ 0.0(3.3)e-06 ]  
      lam_z_SS 0     7.17(10)e-08       [ 4.4(4.4)e-06 ]  
               1    1.652(72)e-07       [ 4.4(4.4)e-06 ]  
   proton_z_PS 0    2.944(60)e-07       [ 0.0(3.3)e-07 ]  
               1    1.235(88)e-06       [ 0.0(3.3)e-07 ]  ***
   proton_z_SS 0     8.59(16)e-08       [ 4.4(4.4)e-07 ]  
               1     2.15(13)e-07       [ 4.4(4.4)e-07 ]  
       xi_z_PS 0    6.278(52)e-07       [ 0.0(3.3)e-06 ]  
               1    2.311(48)e-06       [ 0.0(3.3)e-06 ]  
       xi_z_SS 0  

IndexError: index 0 is out of bounds for axis 0 with size 0

**

## bootstrapping ##

bs resampling and redo fits 

keep track of seed 

gauge average bc approximate integral that way -> physical point

data from configs, resample gauge configs, average them to get result of integral

rand sample m times (gauge field configs) to compute bs sample 

array n x m, average over m (n number of resamples)

each resample n correspond to an integral to be computed by avg over m configurations

https://lsqfit.readthedocs.io/en/latest/lsqfit.html
1) make a large number of “bootstrap copies” of the original input data and prior that differ from each other by random amounts characteristic of the underlying randomness in the original data

2) repeat the entire fit analysis for each bootstrap copy of the data, extracting fit results from each 

3) use the variation of the fit results from bootstrap copy to bootstrap copy to determine an approximate probability distribution (possibly non-gaussian) for the fit parameters and/or functions of them: the results from each bootstrap fit are samples from that distribution.

In [ ]:
fit_ensemble_bs = fm.fit_ensemble(t_range=p_dict['t_range'],
                               n_states=p_dict['n_states'], prior=prior, 
                               nucleon_corr_data=bs_correlators_gv, 
                               axial_fh_num_data=axial_fh_num_data,
                               vector_fh_num_data=vector_fh_num_data)
print(fit_ensemble_bs)

: 

: 

: 

: 

: 

: 

: 

In [ ]:
# Make bs_N fits, one for each of the resampled correlators
fit_keys = fit_out.p.keys()
output = {key : [] for key in fit_keys}

for j, bsfit in enumerate(fit_out.bootstrapped_fit_iter(datalist=bs_correlators_gv)):
    for key in fit_keys:
        # Save the best estimate for the central value 
        # of each parameter of each fit
        p = bsfit.pmean[key]
        output[key].append(p)

# # print results -- should be similar to previous results
table = gv.dataset.avg_data(output, bstrap=True)
print(gv.tabulate(table))

: 

: 

: 

: 

: 

: 

: 

In [ ]:
#print(gv.evalcorr(table[key][:2])[1,0])
# measure of how well multi-dimensional Gaussian distributions(regular fit and bootstrapped fit) agree with each other 
# do their means agree within errors for corresponding elements. 
# note they share the same keys, so just loop over those 
# only parts of fit params that overlap are used 
for key in fit_keys:
    chi2 = gv.chi2(table[key],fit_out.p[key])
gv.fmt_chi2(chi2)

: 

: 

: 

: 

: 

: 

: 